<a href="https://colab.research.google.com/github/saumilthecode/LLM-from-Scratch/blob/main/ShrekGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
%pip install transformers datasets tqdm torch torchvision torchaudio

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset , TensorDataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from tqdm import tqdm
from datasets import load_dataset

torch.backends.cudnn.benchmark = True

# Ensure GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!wget https://raw.githubusercontent.com/saumilthecode/LLM-from-Scratch/refs/heads/main/shrek1%2B3.txt

--2024-09-27 07:44:13--  https://raw.githubusercontent.com/saumilthecode/LLM-from-Scratch/refs/heads/main/shrek1%2B3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 313093 (306K) [text/plain]
Saving to: ‘shrek1+3.txt’

shrek1+3.txt        100%[===================>] 305.75K  --.-KB/s    in 0.03s   

2024-09-27 07:44:13 (11.7 MB/s) - ‘shrek1+3.txt’ saved [313093/313093]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
! pip install huggingface_hub
! pip install transformers
! pip install sentencepiece

device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 64
block_size = 128
max_iters = 10000
learning_rate = 1e-4
eval_iters = 50
n_embd = 512
n_head = 8
n_layer = 6
dropout = 0.1


print(device)

cuda


In [ ]:
with open('shrek1+3.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()  # Set model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)  # Move tensor to GPU
        for k in range(eval_iters):
            # Get batch data from DataLoader
            for X, Y in (train_loader if split == 'train' else val_loader):
                logits, loss = model(X.to(device), Y.to(device))
                losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  # Set model back to training mode
    return out




In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)  # Set bias=False
        self.query = nn.Linear(n_embd, head_size, bias=False)  # Set bias=False
        self.value = nn.Linear(n_embd, head_size, bias=False)  # Set bias=False
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd, bias=False)  # Set bias=False
        self.dropout = nn.Dropout(dropout)

class FeedForward(nn.Module):  # Corrected class name
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd, bias=False),  # Set bias=False
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd, bias=False),  # Set bias=False
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)  # Changed here to FeedForward
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)



In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler


# Create TensorDatasets for training and validation
train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

# Use DataLoader with multiple workers
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)  # Adjust num_workers based on your CPU cores
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


accumulation_steps = 4  # Adjust this value as per your needs
scaler = GradScaler()
optimizer.zero_grad(set_to_none=True)  # Initialize optimizer

for iter in range(max_iters):
    print(f"Iteration {iter}")

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # Load batch data from the DataLoader
    for xb, yb in train_loader:  # Use the DataLoader here
        with autocast():
            logits, loss = model(xb.to(device), yb.to(device))  # Move data to the appropriate device
            loss = loss / accumulation_steps  # Normalize loss for accumulation

        scaler.scale(loss).backward()

        if (iter + 1) % accumulation_steps == 0:
            scaler.step(optimizer)  # Update model parameters
            scaler.update()  # Update scaler for mixed precision
            optimizer.zero_grad(set_to_none=True)  # Reset gradients after step

# Print the final loss after training
print(loss.item())
print(prof.key_averages().table(sort_by="cuda_time_total"))


In [ ]:
prompt = 'battle of midway'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

ONE MASSIVE HUNKA CODE


In [ ]:
# Install necessary libraries
!pip install transformers datasets tqdm torch torchvision torchaudio sentencepiece huggingface_hub

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from tqdm import tqdm
from datasets import load_dataset

torch.backends.cudnn.benchmark = True

# Ensure GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
batch_size = 64
block_size = 128
max_iters = 10000
learning_rate = 1e-4
eval_iters = 50
n_embd = 512
n_head = 8
n_layer = 6
dropout = 0.1

print(f"Device: {device}")

# Load and process text data
with open('shrek1+3.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode dataset
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# Create Dataset and DataLoader
train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Function to generate batches
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Loss estimation
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        loader = train_loader if split == 'train' else val_loader
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

# Model components
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B, T, head_size)
        q = self.query(x)  # (B, T, head_size)
        v = self.value(x)  # (B, T, head_size)

        # Compute attention scores
        wei = q @ k.transpose(-2, -1) * (1.0 / (k.shape[-1] ** 0.5))  # scaled dot-product attention
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # apply mask
        wei = torch.nn.functional.softmax(wei, dim=-1)  # attention weights
        wei = self.dropout(wei)

        # Weighted sum of values
        out = wei @ v  # (B, T, head_size)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd, bias=False),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd, bias=False),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets[:, :T].contiguous().view(B * T)
            loss = nn.CrossEntropyLoss()(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self(index_cond)
            logits = logits[:, -1, :]
            probs = torch.nn.functional.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

# Instantiate model
model = GPTLanguageModel(vocab_size)
model = model.to(device)

# Optimizer and AMP scaler
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scaler = torch.cuda.amp.GradScaler()

# Training loop
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    x_batch, y_batch = get_batch('train')
    with torch.cuda.amp.autocast():
        logits, loss = model(x_batch, y_batch)

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()

# Final testing - Text generation
prompt = 'battle of midway'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
generated_text = decode(model.generate(context, max_new_tokens=100)[0].tolist())
print(generated_text)


Device: cuda


<ipython-input-5-9a3ffbbe29cb>:201: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


step: 0, train loss: 4.496, val loss: 4.480


<ipython-input-5-9a3ffbbe29cb>:210: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


step: 50, train loss: 2.027, val loss: 1.756
step: 100, train loss: 1.896, val loss: 1.662
step: 150, train loss: 1.800, val loss: 1.552
step: 200, train loss: 1.695, val loss: 1.450
step: 250, train loss: 1.647, val loss: 1.393
step: 300, train loss: 1.533, val loss: 1.308
step: 350, train loss: 1.466, val loss: 1.225
step: 400, train loss: 1.410, val loss: 1.200
step: 450, train loss: 1.360, val loss: 1.143
step: 500, train loss: 1.307, val loss: 1.098
step: 550, train loss: 1.266, val loss: 1.077
step: 600, train loss: 1.248, val loss: 1.052
step: 650, train loss: 1.183, val loss: 1.018
step: 700, train loss: 1.164, val loss: 0.996
step: 750, train loss: 1.121, val loss: 0.977
step: 800, train loss: 1.115, val loss: 0.952
step: 850, train loss: 1.093, val loss: 0.947
step: 900, train loss: 1.080, val loss: 0.924
step: 950, train loss: 1.052, val loss: 0.903
step: 1000, train loss: 1.026, val loss: 0.915
step: 1050, train loss: 1.027, val loss: 0.884
step: 1100, train loss: 0.998, va

In [ ]:
prompt = 'shrek'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
generated_text = decode(model.generate(context, max_new_tokens=10)[0].tolist())
print(generated_text)



shrek and at th


In [ ]:
model_save_path = "gpt_model.pt"
torch.save(model.state_dict(), model_save_path)
# Save the model weights
torch.save(model.state_dict(), 'custom_gpt_model.pth')


In [ ]:
pip install huggingface_hub


In [ ]:
from huggingface_hub import login
from google.colab import userdata
userdata.get('HF')

# Log in to Hugging Face
login("HF")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


ValueError: Invalid token passed!

In [ ]:
from huggingface_hub import HfApi, Repository

# Create a repository on Hugging Face
repo_name = "saumil707/SHREKGPT"  # Choose a unique name for your model
api = HfApi()
api.create_repo(repo_name)

# Push the model
repo = Repository(local_dir=repo_name, clone_from=repo_name)
repo.git_add()
repo.git_commit("Initial commit of the model")
repo.git_push()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/saumil707/SHREKGPT into local empty directory.


OSError: Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@2e05bcb8600f.(none)')


SyntaxError: invalid syntax (<ipython-input-22-a6a3ea0d01f8>, line 2)